In [1]:
import torch
import numpy as np

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

WIDTH = 395
HEIGHT = 860

train_data = np.load('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/training_data.npy',allow_pickle=True)


cuda:0


In [2]:
X=np.zeros([13000,395,860], dtype=np.uint16)
test_x=np.zeros([4000,395,860], dtype=np.uint16)
for i in range(len(train_data)):
    if i < 13000:
        X[i]=train_data[i,0][80:475,50:910]
    else:
        test_x[i-13000]=train_data[i,0][80:475,50:910]

In [3]:
import pandas as pd

val=pd.read_csv('C:/Users/Ant Pc/GitHub/AI-in-Mfg---Automated-Driving/Aaftab/choice.csv')
print(val.head())
Y=val.iloc[0:13000,0]
test_y=val.iloc[13000:,0]

   0
0  0
1  0
2  0
3  0
4  0


In [4]:
print(X.shape)
print(test_x.shape)
print(Y.shape)
print(test_y.shape)

(13000, 395, 860)
(4000, 395, 860)
(13000,)
(4000,)


In [5]:
import torch
import torch.nn as nn

model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=False)
model.eval()

Using cache found in C:\Users\Ant Pc/.cache\torch\hub\pytorch_vision_v0.6.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [6]:
model = model.cuda()
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

27161264


In [ ]:
#trainingloop
import cv2
torch.cuda.empty_cache()
import torch.optim as optim

lr = 0.001
batchsize=2

batches=13000/batchsize

epochs=30

# create your optimizer
optimizer = optim.Adam(model.parameters(), lr)

losse=0
for i in range(epochs):
    
    print(losse,i)
    losse=0
    for j in range(int(batches)):
        
        #forward pass
           
        a=np.array([cv2.cvtColor(i,cv2.COLOR_GRAY2RGB) for i in X[j:j+batchsize,:,:]])
       
        trdttensor=torch.from_numpy(a.reshape([batchsize,3,395,860]).astype('int')).type(dtype)
        
        resultstensor=torch.from_numpy((Y[j:j+batchsize][:].astype('int')).values).type(dtype)
                
        out=model(trdttensor).type(dtype)
        #compute loss
        criterion = nn.CrossEntropyLoss()
        d=nn.Softmax(dim=1)
        loss = criterion(d(out),resultstensor.long().squeeze()).type(dtype)
        
        
        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        optimizer.zero_grad()
        #use optimiser to update
        optimizer.step()
        losse=losse+loss.detach().cpu().numpy()
        if j%10==0:
            print(j)
        if j%1000==0:
            print(losse,j)

0 0
0
6.909472465515137 0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
6916.381937980652 1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
13825.854403495789 2000
2010
2020
2030
2040
2050
2060
2070
